## Imports

In [19]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd

from pandas_geojson import to_geojson, write_geojson
from geopy.geocoders import GoogleV3

## Data Read-in

In [2]:
df = pd.read_csv("South Florida projects registered in NY - map - Sheet1.csv")

## Data work

In [3]:
df['full_address'] = df['address_part_1'] + ' ' + df['address_part_2']

In [6]:
df['Sales and marketing brokerage'].unique()

array(['TBD: JDS is in the process of replacing Official, sources say',
       'Official', 'Douglas Elliman', 'Serhant',
       'IMI Worldwide Properties', 'The Inside Network',
       'Cervera Real Estate ', 'In-house', 'Oculus Realty ',
       'One Sotheby’s International Realty',
       'Fortune International Group', "Douglas Elliman's Dan Teixeira ",
       'Related Realty', 'OneWorld Properties'], dtype=object)

In [ ]:

       'Related Realty', 'OneWorld Properties

In [11]:
map_dict = {
    'TBD: JDS is in the process of replacing Official, sources say':'#FF5733', # (Burnt Orange)
    'Official':'#FF5733', # (Burnt Orange)
    'Douglas Elliman':'#3357FF', # (Royal Blue)
    'Serhant':'#FF33A6', # (Hot Pink)
    'IMI Worldwide Properties':'#FFD733', # (Golden Yellow)
    'The Inside Network':'#33FFF7', # (Cyan)
    'Cervera Real Estate':'#FF5733', # (Tomato Red)
    'In-house':'#8D33FF', # (Purple)
    'Oculus Realty':'#FF8D33', # (Coral)
    'One Sotheby’s International Realty':'#33FF8D', # (Mint Green)
    'Fortune International Group':'#5733FF', # (Indigo)
    "Douglas Elliman's Dan Teixeira":'#3357FF', # (Royal Blue)
    'Related Realty':'#FF33D7', # (Magenta)
    'OneWorld Properties':'#33A6FF' # (Sky Blue)
}

df_map_colors = pd.DataFrame(list(map_dict.items()), columns=['Company', 'Color'])

In [14]:
df.columns

Index(['address_part_1', 'address_part_2', 'Project name', 'Developers',
       'Sales and marketing brokerage', 'full_address'],
      dtype='object')

In [15]:
pd.merge?

In [17]:
df_merge = pd.merge(left=df, right=df_map_colors,
                    left_on='Sales and marketing brokerage', right_on='Company')

In [18]:
df_merge

,address_part_1,address_part_2,Project name,Developers,Sales and marketing brokerage,full_address,Company,Color
0,888 BRICKELL AVENUE,"MIAMI, FL 33131",888 Brickell Miami,JDS Development Group,TBD: JDS is in the process of replacing Offici...,"888 BRICKELL AVENUE MIAMI, FL 33131",TBD: JDS is in the process of replacing Offici...,#FF5733
1,9110 WEST BAY HARBOR DRIVE,"BAY HARBOR, FL 33154",Indian Creek Residences & Yacht Club,Landau Properties,Official,"9110 WEST BAY HARBOR DRIVE BAY HARBOR, FL 33154",Official,#FF5733
2,151 N. SEABREEZE BLVD.,"FORT LAUDERDALE, FL 33304",Selene Oceanfront Residences,Kolter Group,Douglas Elliman,"151 N. SEABREEZE BLVD. FORT LAUDERDALE, FL 33304",Douglas Elliman,#3357FF
3,100 E. LAS OLAS BLVD,"FORT LAUDERDALE, FL 33301",100 Las Olas,Kolter Group,Douglas Elliman,"100 E. LAS OLAS BLVD FORT LAUDERDALE, FL 33301",Douglas Elliman,#3357FF
4,1133 SW 2ND AVENUE,"MIAMI, FL 33130",Mercedes-Benz Places,JDS Development Group,Serhant,"1133 SW 2ND AVENUE MIAMI, FL 33130",Serhant,#FF33A6
5,1309 SOUTH FLAGLER DRIVE,"WEST PALM BEACH, FL 33401",Forte Residences,Two Roads Development,Douglas Elliman,"1309 SOUTH FLAGLER DRIVE WEST PALM BEACH, FL 3...",Douglas Elliman,#3357FF
6,1901 SOUTH OCEAN BOULEVARD,"DELRAY BEACH, FL 33483",Ocean Delray,"U.S. Construction, National Realty Investment ...",IMI Worldwide Properties,"1901 SOUTH OCEAN BOULEVARD DELRAY BEACH, FL 33483",IMI Worldwide Properties,#FFD733
7,1055 93RD STREET,"BAY HARBOR, FL 33154",Solina Bay Harbor,RedHoek+,The Inside Network,"1055 93RD STREET BAY HARBOR, FL 33154",The Inside Network,#33FFF7
8,90 NE 11TH STREET,"MIAMI, FL 33132",E11even Residences Beyond,"PMG, E11even Partners",In-house,"90 NE 11TH STREET MIAMI, FL 33132",In-house,#8D33FF
9,20 NE 11TH STREET,"MIAMI, FL 33132",E11even Hotel & Residences,"PMG, E11even Partners",In-house,"20 NE 11TH STREET MIAMI, FL 33132",In-house,#8D33FF


## Google Maps and geolocater setup

In [20]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [23]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df_merge['geocoded'] = df_merge['full_address'].apply(geocode)

In [24]:
df_merge['geocoded'] = df_merge['geocoded'].astype(str)
df_merge[['lat', 'lon']] = df_merge['geocoded'].str.strip('()').str.split(', ', expand=True)
df_merge['lat'] = df_merge['lat'].astype(float)
df_merge['lon'] = df_merge['lon'].astype(float)

In [29]:
df_merge = df_merge.rename(columns={'full_address':'Address'})

In [30]:
df_merge.columns

Index(['address_part_1', 'address_part_2', 'Project name', 'Developers',
       'Sales and marketing brokerage', 'Address', 'Company', 'Color',
       'geocoded', 'lat', 'lon'],
      dtype='object')

In [34]:
geo_json = to_geojson(df=df_merge, lat='lat', lon='lon',
                     properties=['Project name','Address','Developers','Sales and marketing brokerage',
                                'Color'])
write_geojson(geo_json, filename='SoFlaCondoMarketingNY.geojson', indent=4)

## Map URL Snagger

In [35]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/SoFlaDevelopers_NY_Marketed_050724'
print(base_name)

https://trd-digital.github.io/trd-news-interactive-maps/SoFlaDevelopers_NY_Marketed_050724
